# Action Client node with Jupyter 

### Import code

In [1]:
import rospy
import ipywidgets as widgets
from nav_msgs.msg import Odometry
import actionlib
import actionlib.msg
import assignment_2_2023.msg
from assignment_2_2023.msg import Robotinfo, Robotinfofeet
from actionlib_msgs.msg import GoalStatus
from assignment_2_2023.srv import CancelGoal, CancelGoalResponse
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

### New Imports

### Global variables

In [2]:
subPos = None  
clienttar = None
servCancGoal = None

### pos_callback

In [ ]:
def pos_callback(msg):
    global subPos

### Init of the variouse variables

In [4]:
"""
    This function initializes the nodes and calls the action client function.
    This also initializes the publisher and subscriber for the robot information, the service for the goal canceling and the action to reach the goal.
"""
global subPos, clienttar, servCancGoal

# Wait for gazebo to be up and running
time.sleep(1)

# Initialize the node
rospy.init_node('user_interface')

# Create the service Client
clienttar = actionlib.SimpleActionClient('/reaching_goal',assignment_2_2023.msg.PlanningAction)
""" Client for the action server reaching_goal
"""

# Service to cancel the goal
servCancGoal = rospy.ServiceProxy('/cancel_goal', CancelGoal) #  goal_canceling_service va aggiunto?
""" Service to cancel the goal
"""

# Call the subscriber from Odom for position and velocity
subPos = rospy.Subscriber('/odom', Odometry, pos_callback)
""" Subscriber for the position
"""

' Publisher for the robot information in feet\n'

### Insert number for the target

va completamente cambiata e inseriti i widgets

In [5]:
def insertNumber():
    """
    Function to take the input from the user to know the coordinates of the goal

    Returns
    -------
    inpx : float
        x coordinate of the goal    
    inpy : float    
        y coordinate of the goal
    """
    while(1):
        inpx = input("x: ")
        try:
            inpx = float(inpx)
            break
        except:
            print("Please insert a valid number")
    while(1):
        inpy = input("y: ")
        try:
            inpy = float(inpy)
            break
        except:
            print("Please insert a valid number")
    return inpx, inpy

### goal cancelling service

In [6]:
def goal_canceling_service(req):
    """
    Service to cancel the goal
    
    Args:
    req (CancelGoal): request to cancel the goal
    """
    
    global servCancGoal

    rospy.wait_for_service('/cancel_goal')

    response = CancelGoalResponse()

    
    try: 
        clienttar.cancel_goal() 
        response.stat = "Done"
    except:
        response.stat = "Error"
    
    return response

### Action Client definition

In [7]:
"""
Function to implement the action client
"""

global clienttar, servCancGoal

# Wait until the action server is up and running.
clienttar.wait_for_server()


# User interface (from here modify)
print("Hi, welcome to the robot planner simulation")
print("Please choose the position you want to reach with the robot")
print("Insert the coordinates to reach as ")

# Create a goal to send (to the action server)
goal = assignment_2_2023.msg.PlanningGoal()

# Implements the coordinates acquisition
inpx, inpy = insertNumber()

# Update the goal
goal.target_pose.pose.position.x = inpx
goal.target_pose.pose.position.y = inpy

# Sending the goal
clienttar.send_goal(goal)
# print("You sent the goal with: X = ", goal.target_pose.pose.position.x," Y = ", goal.target_pose.pose.position.y)

while not rospy.is_shutdown(): 
    # Input from the user
    choice = input("Do you want to insert a new goal? (y/n) ")
    if (choice == 'y'):
        # Implements the coordinates acquisition
        inpx, inpy = insertNumber()

        # Update the goal
        goal.target_pose.pose.position.x = inpx
        goal.target_pose.pose.position.y = inpy

        # Send the goal
        clienttar.send_goal(goal)
        print("You sent the goal with: X = ", goal.target_pose.pose.position.x," Y = ", goal.target_pose.pose.position.y)
    else:
        choice = input("Do you want to cancel the previouse goal and restore ? (y/n) ")
        if (choice == 'y'):
            if (clienttar.get_state() == GoalStatus.ACTIVE):
                print("You canceled the goal with: X = ", goal.target_pose.pose.position.x," Y = ", goal.target_pose.pose.position.y)
                clienttar.cancel_goal() 
            else:
                print("The goal has already finished or it was already canceled")
        else:
            print("No Operations has been done")

Hi, welcome to the robot planner simulation
Please choose the position you want to reach with the robot
Insert the coordinates to reach as 
Please insert a valid number
You sent the goal with: X =  7.0  Y =  7.0
